In [1]:
import nltk
import requests
import warnings
from sqlalchemy import text
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer

import sqlite3

# Data manipulation and analysis
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sqlalchemy.orm import sessionmaker
import matplotlib.dates as mdates

from datetime import datetime
from datetime import timedelta

# Financial data extraction
import yfinance as yf

# SQL Alchemy for database interaction
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session

# Additional libraries for analysis or visualization
import matplotlib.pyplot as plt
import seaborn as sns
from config import api_key

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [12]:
engine = create_engine('sqlite:///stock_market_analysis.sqlite')
session = Session(engine)

tables = inspect(engine).get_table_names()
print(tables)
print(' ')
for table in tables:
    print(pd.read_sql(table, engine).head())

['Average_Sentiment_Score', 'stock_history']
 
   index  Average Sentiment Score
0   MSFT                 0.045168
1   AAPL                 0.038288
2   SHOP                 0.113021
        Date        Open        High         Low       Close   Adj Close  \
0 2024-01-02  373.859985  375.899994  366.769989  370.869995  370.185425   
1 2024-01-03  369.010010  373.260010  368.510010  370.600006  369.915924   
2 2024-01-04  370.670013  373.100006  367.170013  367.940002  367.260834   
3 2024-01-05  368.970001  372.059998  366.500000  367.750000  367.071198   
4 2024-01-08  369.299988  375.200012  369.010010  374.690002  373.998383   

     Volume Ticker  
0  25258600   MSFT  
1  23083500   MSFT  
2  20901500   MSFT  
3  20987000   MSFT  
4  23134000   MSFT  


In [2]:
def load_data_to_db(data, ticker):
    """
    Loads transformed data into the SQLite database.
    """
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    data.to_sql(ticker, con=engine, if_exists='replace', index=True)

In [3]:

def fetch_all_stock_data(tickers, start='2023-04-01', end='2024-04-01'):
    stock_data = {}
    for ticker in tickers:
        print(f"Fetching data for {ticker}")
        stock_data[ticker] = yf.download(ticker, start, end)

    for key in stock_data.keys():
        stock_data[key]['Ticker'] = key

    df = pd.concat(stock_data.values())

    load_data_to_db(df, 'stock_history')

    return '<h1>SQLite Database was updated</h1>'

In [4]:
tickers = ['zm','msft','shop']
stock_data = fetch_all_stock_data(tickers)


Fetching data for zm


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Fetching data for msft
Fetching data for shop


[*********************100%%**********************]  1 of 1 completed


In [9]:
engine = create_engine('sqlite:///stock_market_analysis.sqlite')
inspect(engine).get_table_names()

['Average_Sentiment_Score', 'stock_history']

In [34]:
df1 = pd.DataFrame(session.execute(text('SELECT * FROM Average_Sentiment_Score')).all())
df2 = pd.DataFrame(session.execute(text('SELECT * FROM stock_history')).all())

df3 = df1.merge(df2, left_on='index',right_on='Ticker').drop('index',axis=1)

df3 = df3[['Date','Ticker', 'Average Sentiment Score', 'Open', 'High', 'Low', 'Close',
       'Adj Close', 'Volume']]
df3.Date = df3.Date.str.replace('\s.*','',regex=True)
df3.sort_values('Date',inplace=True)

df3.to_json(orient='columns')

'{"Date":{"0":"2024-01-02","61":"2024-01-02","122":"2024-01-02","1":"2024-01-03","62":"2024-01-03","123":"2024-01-03","2":"2024-01-04","63":"2024-01-04","124":"2024-01-04","3":"2024-01-05","64":"2024-01-05","125":"2024-01-05","65":"2024-01-08","126":"2024-01-08","4":"2024-01-08","127":"2024-01-09","5":"2024-01-09","66":"2024-01-09","128":"2024-01-10","6":"2024-01-10","67":"2024-01-10","129":"2024-01-11","7":"2024-01-11","68":"2024-01-11","130":"2024-01-12","69":"2024-01-12","8":"2024-01-12","70":"2024-01-16","131":"2024-01-16","9":"2024-01-16","71":"2024-01-17","132":"2024-01-17","10":"2024-01-17","72":"2024-01-18","133":"2024-01-18","11":"2024-01-18","134":"2024-01-19","12":"2024-01-19","73":"2024-01-19","13":"2024-01-22","74":"2024-01-22","135":"2024-01-22","14":"2024-01-23","75":"2024-01-23","136":"2024-01-23","76":"2024-01-24","137":"2024-01-24","15":"2024-01-24","138":"2024-01-25","16":"2024-01-25","77":"2024-01-25","139":"2024-01-26","17":"2024-01-26","78":"2024-01-26","140":"202

In [33]:
pd.read_sql('stock_history', engine).to_json(orient='records')


'[{"Date":1680480000000,"Open":73.1699981689,"High":73.7649993896,"Low":72.3700027466,"Close":73.6699981689,"Adj Close":73.6699981689,"Volume":2304700,"Ticker":"zm"},{"Date":1680566400000,"Open":74.1500015259,"High":74.4100036621,"Low":72.8949966431,"Close":73.6399993896,"Adj Close":73.6399993896,"Volume":1812500,"Ticker":"zm"},{"Date":1680652800000,"Open":73.0100021362,"High":73.0599975586,"Low":70.1399993896,"Close":71.0999984741,"Adj Close":71.0999984741,"Volume":3576900,"Ticker":"zm"},{"Date":1680739200000,"Open":70.7149963379,"High":72.0999984741,"Low":69.5599975586,"Close":71.8499984741,"Adj Close":71.8499984741,"Volume":2074400,"Ticker":"zm"},{"Date":1681084800000,"Open":71.1800003052,"High":71.3310012817,"Low":69.6600036621,"Close":71.1699981689,"Adj Close":71.1699981689,"Volume":2137000,"Ticker":"zm"},{"Date":1681171200000,"Open":68.8700027466,"High":70.8000030518,"Low":68.25,"Close":70.0,"Adj Close":70.0,"Volume":4037300,"Ticker":"zm"},{"Date":1681257600000,"Open":71.16999816

In [ ]:

# Merging DataFrames
df3 = df1.merge(df2, left_on='Ticker', right_on='Ticker')
df3.drop('index', axis=1, inplace=True)

# Formatting and selecting data
df3 = df3[['Date', 'Ticker', 'Average Sentiment Score', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
df3['Date'] = pd.to_datetime(df3['Date']).dt.date  # Ensure Date is in YYYY-MM-DD format
df3.set_index('Date', inplace=True)
df3.sort_index(inplace=True)

# Return the formatted data as JSON
df3.to_json(orient='records')

In [12]:
def fetch_all_stock_data(tickers, start='2023-04-01', end='2024-04-01'):
    stock_data = {}
    
    # get_avg_sentiment_scores(tickers)
    
    for ticker in tickers:
        print(f"Fetching data for {ticker}")
        stock_data[ticker] = yf.download(ticker, start, end)

    df = pd.DataFrame(stock_data)
    load_data_to_db(df, 'stock_history')

    return '<h1>Data is now loaded in database!</h1>'

In [16]:
tickers = ['zm','msft','shop']
start = '2023-04-01'
end = '2024-04-01'

In [13]:
fetch_all_stock_data(['zm','msft','shop'],'2023-04-01','2024-04-01')

[*********************100%%**********************]  1 of 1 completed

Fetching data for zm


{'zm':                  Open       High        Low      Close  Adj Close   Volume
 Date                                                                      
 2023-04-03  73.169998  73.764999  72.370003  73.669998  73.669998  2304700
 2023-04-04  74.150002  74.410004  72.894997  73.639999  73.639999  1812500
 2023-04-05  73.010002  73.059998  70.139999  71.099998  71.099998  3576900
 2023-04-06  70.714996  72.099998  69.559998  71.849998  71.849998  2074400
 2023-04-10  71.180000  71.331001  69.660004  71.169998  71.169998  2137000
 ...               ...        ...        ...        ...        ...      ...
 2024-03-22  66.349998  66.410004  65.580002  65.959999  65.959999  1521200
 2024-03-25  65.849998  66.940002  65.349998  66.709999  66.709999  2227000
 2024-03-26  67.000000  67.394997  66.184998  66.349998  66.349998  1642100
 2024-03-27  66.610001  66.959999  65.930000  66.620003  66.620003  1882900
 2024-03-28  66.860001  66.860001  65.330002  65.370003  65.370003  2605800
 
 [24

In [ ]:
# Function to fetch headlines from NewsAPI
def fetch_news(api_key, ticker):
    base_url = "https://newsapi.org/v2/everything"
    params = {
        'q': ticker,             # Search query (ticker symbol)
        'sortBy': 'publishedAt', # Sort by publication date
        'apiKey': api_key        # Your NewsAPI key
    }
    response = requests.get(base_url, params=params)
    articles = response.json().get('articles', [])
    headlines = [article['title'] for article in articles]
    return headlines

In [ ]:
def get_avg_sentiment_scores():
    textual_data = {}
    tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']

    for ticker in tickers:
        headlines = fetch_news(api_key, ticker)
        textual_data[ticker] = headlines

    # Analyzing sentiment for each headline and averaging the scores
    average_sentiment_scores = {}

    for ticker, headlines in textual_data.items():
        sentiments = [TextBlob(headline).sentiment.polarity if headline else 0 for headline in headlines]
        average_sentiment_scores[ticker] = np.mean(sentiments)

    # Extract tickers and sentiment scores
    tickers = list(average_sentiment_scores.keys())
    sentiment_scores = list(average_sentiment_scores.values())

    df = pd.DataFrame(sentiment_scores, tickers,['Average Sentiment Score'])
    
    load_data_to_db(df, 'Average_Sentiment_Score')

In [ ]:
def calculate_daily_return(stock_data):
    """
    Calculates the daily return from the closing prices.
    """
    stock_data['Daily_Return'] = stock_data['Close'].pct_change()
    return stock_data

In [ ]:
def read_data(ticker):
    """
    Reads the data for the specified ticker from the SQLite database.
    """
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    query = f"SELECT * FROM '{ticker}'"
    data = pd.read_sql_query(query, con=engine)
    return data

In [ ]:
def predict_monthly_prices(model, last_known_price, last_known_date, months=36):
    future_prices = []
    future_dates = []
    current_price = last_known_price
    
    for month in range(months):
        # Approximating each month by 21 trading days
        for day in range(21):
            next_input = pd.DataFrame(data=[[current_price]], columns=['Previous Close'])
            current_price = model.predict(next_input)[0]
        
        future_prices.append(current_price)
        # Assuming last_known_date is a datetime object; add roughly 30 days for each month
        last_known_date += timedelta(days=30)
        future_dates.append(last_known_date)
    
    return future_dates, future_prices

In [ ]:
def train_model_for_ticker(ticker_data):
    """
    Trains a LinearRegression model for the given ticker data.
    """
    data = ticker_data[['Close']].copy()
    data['Previous Close'] = data['Close'].shift(1)
    data.dropna(inplace=True)  # Drop the first row which now contains NaN

    X = data[['Previous Close']]  # Features
    y = data['Close']  # Target

    model = LinearRegression()
    model.fit(X, y)  # Training the model
    return model

In [ ]:
# Objective Function (Negative Sharpe Ratio)
def neg_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate=0.01):
    p_var = np.dot(weights.T, np.dot(cov_matrix, weights))
    p_ret = np.dot(weights, expected_returns)
    return -(p_ret - risk_free_rate) / np.sqrt(p_var)

In [ ]:
def analyze_sentiment(text):
    """Analyze the sentiment of a text and return polarity and subjectivity."""
    analysis = TextBlob(text)
    return analysis.sentiment.polarity, analysis.sentiment.subjectivity

In [ ]:
def main():
    tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']

    for ticker in tickers:
        print(f"Fetching news for: {ticker}")
        articles = fetch_news(api_key, ticker)
        if articles['status'] == 'ok':
            for article in articles['articles'][:5]:  # Show only the first 5 articles for brevity
                description = article.get('description') or article.get('title')
                polarity, subjectivity = analyze_sentiment(description)
                print(f"Ticker: {ticker}, Title: {article['title']}")
                print(f"Sentiment Polarity: {polarity:.2f}, Sentiment Subjectivity: {subjectivity:.2f}")
                print(f"URL: {article['url']}\n")
    
    return articles

In [ ]:
def fetch_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="5y")
    return data['Close']

In [ ]:
def calculate_parameters(tickers):
    prices = {ticker: fetch_data(ticker) for ticker in tickers}
    prices = pd.DataFrame(prices)
    daily_returns = prices.pct_change()
    annual_mean_returns = daily_returns.mean() * 252
    annual_std_devs = daily_returns.std() * np.sqrt(252)
    return {ticker: {'mean': annual_mean_returns[ticker], 'std': annual_std_devs[ticker]}
            for ticker in tickers}

In [1]:
def calculate_expected_returns(tickers):
    prices = pd.DataFrame({ticker: fetch_data(ticker) for ticker in tickers})
    daily_returns = prices.pct_change()
    average_daily_returns = daily_returns.mean()
    annual_expected_returns = average_daily_returns * 252  # Convert to annual returns
    return annual_expected_returns

In [13]:
get_avg_sentiment_scores()

In [18]:
textual_data = {}
tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']

for ticker in tickers:
    headlines = fetch_news(api_key, ticker)
    textual_data[ticker] = headlines

# Analyzing sentiment for each headline and averaging the scores
average_sentiment_scores = {}

for ticker, headlines in textual_data.items():
    sentiments = [TextBlob(headline).sentiment.polarity if headline else 0 for headline in headlines]
    average_sentiment_scores[ticker] = np.mean(sentiments)

# Extract tickers and sentiment scores
tickers = list(average_sentiment_scores.keys())
sentiment_scores = list(average_sentiment_scores.values())

df = pd.DataFrame(sentiment_scores, tickers,['Average Sentiment Score'])

load_data_to_db(df, 'Average_Sentiment_Score')

In [19]:
df

,Average Sentiment Score
MSFT,0.033303
CRM,0.022510
CRWD,0.091855
ZM,0.020036
SHOP,0.168199
AAPL,0.019136


In [34]:
df.to_sql(ticker, con=engine, if_exists='replace', index=True)

6

In [24]:
ticker

'AAPL'

In [23]:
df

,Average Sentiment Score
MSFT,0.033303
CRM,0.022510
CRWD,0.091855
ZM,0.020036
SHOP,0.168199
AAPL,0.019136


In [38]:
session = Session(engine)

In [45]:
pd.read_sql('SELECT * FROM CRM', engine)

,date,open,high,low,close,volume


In [43]:
session.execute('SELECT * FROM Average_Sentiment_Score').fetchall()

[('MSFT', 0.0333031746031746),
 ('CRM', 0.022509740259740257),
 ('CRWD', 0.09185533910533909),
 ('ZM', 0.0213996558996559),
 ('SHOP', 0.12842676767676767),
 ('AAPL', 0.01913567821067821)]

In [8]:
con = sqlite3.connect(engine)

TypeError: expected str, bytes or os.PathLike object, not Engine

In [32]:
read_data('Average_Sentiment_Score').to_json()

'{"index":{"0":"MSFT","1":"CRM","2":"CRWD","3":"ZM","4":"SHOP","5":"AAPL"},"Average Sentiment Score":{"0":0.0333031746,"1":0.0225097403,"2":0.0918553391,"3":0.0200360195,"4":0.1681994949,"5":0.0191356782}}'

In [16]:
if request.args.get('api_key') and request.args['api_key'] == '4497341d87444fddb894bbc281c9f9f8':
    return f(*args, **kwargs)
else:
    return jsonify({"error": "Authentication required"}), 403


SyntaxError: 'return' outside function (828057532.py, line 2)